In [1]:
%reload_ext autoreload
%autoreload 2

import sys,os,time,json
from scipy.interpolate import interp1d
import numpy as np
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.getcwd(), '../'))
# from hardware.slm import SLM
from hardware.camera import NITCam
from hardware.waveshaper import Waveshaper
from hardware.yokogawa_osa import YokogawaOSA
from hardware.elliptec import ElliptecDualPositioner
from hardware.rs_oscilloscope import RsOSC
from hardware.rs_powersupply_hmp4040 import RsHMP4040
from hardware.thorlabs_edfa300p import ThorlabsEDFA
from hardware.keysight_psg import KeysightPSG
from experiments.speckle_collector_with_comb import SpeckleCollectorWithComb


def thz2nm(THz):
    return 299792.458/THz

%matplotlib qt

from experiments.speckle_collector_with_comb import SpeckleCollectorWithComb

starting = False

In [2]:
starting = True

In [3]:
if starting:
    psg = KeysightPSG()
    psg.connect()

    hmp = RsHMP4040()
    hmp.connect()

    edfa = ThorlabsEDFA()
    edfa.connect()

    psg.set_output(1)
    time.sleep(1)
    hmp.turn_on_selected_channels()

    edfa = ThorlabsEDFA()
    edfa.connect()
    edfa.set_laser_diode_current(100)
    # edfa.enable_laser()

    ws = Waveshaper()
    # ws.connect() # Not necessary for waveshaper

    osa = YokogawaOSA()
    osa.connect()

    camera = NITCam()
    camera.bitDepth = 14
    camera.exposure_time = 2000
    # camera.analog_gain = "High"
    # camera.FPS = 10
    camera.frame_width = 256#424#432
    camera.frame_height = 256#320#424#432
    camera.offset_x = 32#320#496
    camera.offset_y = 0#252#212
    # cam.connect()

    stage = ElliptecDualPositioner()
    stage.connect()

    osc = RsOSC()
    osc.connect()

    def callback_osc():
        osc.single_and_wait()
        time.sleep(0.1)
        return osc.get_trace(channel=1, plot=False)
    
    osc.set_horizontal_scale_sec(0.02)
    osc.set_sample_points(10000)
    osc.set_vertical_offset_volt(1, 0)
    osc.set_vertical_position_div(1,-4.9)
    osc.set_vertical_scale_volt(1, 0.4/10)

    sc = SpeckleCollectorWithComb(camera=camera, ws=ws)
    sc.comb_fsr_ghz = 25
    sc.comb_min_wl_nm = 1532
    sc.comb_max_wl_nm = 1564

    
    # load the above peak array from json
    with open(r'C:\Users\think\Desktop\comb_peak.json', 'r') as f:
        data = json.load(f)
    osa_thz = np.array(data['osa_thz'])
    osa_pwr = np.array(data['osa_pwr'])
    f = interp1d(osa_thz, osa_pwr, kind='linear', fill_value=-np.inf, bounds_error=False)
    def get_att_from_thz(thz, max_input = -35):
        return max(f(thz) - max_input,0)

trying to connect to 192.168.1.8 : 5025
connected to server 192.168.1.8:5025


AttributeError: 'KeysightPSG' object has no attribute 'inst'

In [5]:
sc.pass_one_comb_line(10, passband_atten=5)

In [4]:
camera.connect()
camera.setNUCfile(r'C:\Users\think\Desktop\NUCFactory_2000us.yml') 

In [5]:
sc.comb_fsr_ghz = 25
ii = 1


for ii in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]:
    def get_filename(filename_identify):
        return r'Z:\Maodong\Projects\Speckle_with_Comb\CombPattern_20250102\dataset%d_%s_one_EDFA_1umMMFtocollect_2MMFand1500DF_collect_with_new_collimator\pattern_%d_%d_%dGHz_%ds' % (ii, filename_identify, sc.comb_min_wl_nm, sc.comb_max_wl_nm, sc.comb_fsr_ghz, osc.get_horizontal_scale_sec()*12)


    stage.set_position('open')
    filename_identify = 'withoutobj'
    osc.set_vertical_scale_volt(channel=1, scale=0.35/10)
    filename = get_filename(filename_identify)
    time.sleep(1)
    sc.scan_pattern(filename=filename, callback=callback_osc, get_bandpass_atten_func=get_att_from_thz)

    stage.set_position('block')
    filename_identify = 'withobj'
    osc.set_vertical_scale_volt(channel=1, scale=0.25/10)
    filename = get_filename(filename_identify)
    time.sleep(1)
    sc.scan_pattern(filename=filename, callback=callback_osc, get_bandpass_atten_func=get_att_from_thz)


  6%|▌         | 9/160 [04:16<1:11:39, 28.47s/it]


KeyboardInterrupt: 

In [57]:
edfa.set_laser_diode_current(100)

'\x1b[2K\rsloc 100\r\nCommand description: SET LASER DIODE OPERATING CURRENT\r\n   current is set to \x1b\x1b100.000%\r\n\x1b\x1b[2K'

In [58]:
osa.get_trace('f')

(array([1510.  , 1510.01, 1510.02, ..., 1589.98, 1589.99, 1590.  ]),
 array([-210.       , -210.       ,  -81.9338957, ...,  -66.6731858,
        -210.       ,  -71.8144576]))

In [35]:
stage.set_position('block')
edfa.disable_laser()
edfa.set_laser_diode_current(0)


'\x1b[2K\rsloc 0\r\nCommand description: SET LASER DIODE OPERATING CURRENT\r\n   current is set to \x1b\x1b0.000%\r\n\x1b\x1b[2K'

In [37]:
hmp.connect()

trying to connect to 192.168.1.8 : 5025
connected to server 192.168.1.8:5025


True

In [38]:
time.sleep(1)
hmp.turn_off_selected_channels()


client send: OUTP:GEN 0


In [41]:
hmp.get_active_channel()

client send: INST OUT1
client send: OUTP?
received: 0

client send: INST OUT2
client send: OUTP?
received: 0

client send: INST OUT3
client send: OUTP?
received: 0

client send: INST OUT4
client send: OUTP?
received: 0



['0\n', '0\n', '0\n', '0\n']

In [42]:
time.sleep(1)
psg.set_output(0)

In [43]:
edfa.get_status()

'\x1b[2K\rstat\r\nCommand description: STATUS REGISTER\r\n\r\n  Status Register Value = 0x000E \r\n\n  Interlock  : \x1b\x1bClosed\x1b \r\n  TEC0 temp  : \x1b\x1bGood\x1b \r\n  TEC1 temp  : \x1b\x1bGood\x1b \r\n  Temp stable: \x1b\x1bStable\x1b \r\n  Temp fault : \x1b\x1bNo Fault\x1b \r\n  LASER      : \x1b\x1bOFF\x1b\x1b\x1b \r\n  LOS Status : \x1b\x1bGood\x1b\x1b\x1b \r\n\x1b[2K'

In [45]:
sc.pass_one_comb_line(10)

In [50]:
stage.set_position('block')

In [44]:
camera.disconnect()

In [53]:
plt.figure()
plt.imshow(camera.capture_single_frame(), cmap='gray')

In [ ]:
# osa.write_trace('b')
# osa.run_and_wait()
# osa_nm, osa_pwr = osa.get_trace('b')
# osa_thz = np.array([thz2nm(nm) for nm in osa_nm])
# from scipy.signal import find_peaks
# peaks, _ = find_peaks(osa_pwr, height=-40)# height=-47.5)
# plt.figure()
# plt.plot(osa_thz, osa_pwr)
# plt.plot(osa_thz[peaks], osa_pwr[peaks], 'x')

# osa.fix_trace('b')
# osa.blank_trace('b')